## separate crowd and commentary


In [ ]:
# !pip install -U denoiser
from denoiser import pretrained
from denoiser.dsp import convert_audio
from IPython.display import Audio, display
import torch
import torchaudio

from pydub import AudioSegment
import matplotlib.pyplot as plt
from scipy.io import wavfile
from tempfile import mktemp
import numpy as np
import os
import mutagen


In [ ]:
files = ['WOL_vs_ARS_2022-11-12.mp3']
path_to_raw = '../data/audio/raw/'
path_to_mono = '../data/audio/mono/'
path_to_clean = '../data/audio/clean/'
path_to_background = '../data/audio/background/'

#function to find all the file names in raw that are not in clean

def diff_files(dir1, dir2):
    # Get a list of all files in both directories
    files1 = set(os.listdir(dir1))
    files2 = set(os.listdir(dir2))

    # Find the files that are in dir1 but not in dir2
    diff = files1 - files2

    return [file for file in list(diff) if file[-4:]=='.mp3']

file_list = diff_files(path_to_raw,path_to_clean)
print(file_list)

# run through all uncleaned files 
files = file_list

In [ ]:
%%time

for filename in files:
    
    # read in raw as mono
    mp3_audio = AudioSegment.from_file(path_to_raw+filename, format="mp3")  # read mp3
    tempfile1 = mktemp('.wav')
    mp3_audio.set_channels(1).export(tempfile1, format="wav")
    
    
    # read in raw as wav file
    raw_rate, data = wavfile.read(tempfile1)
    # save to mono file
    wavfile.write(path_to_mono+filename, raw_rate,data)
    
    #split into 10 minute segments
    delta_t = (raw_rate * 60 * 10)
    num_splits = len(data)//delta_t
    print('splitting file',filename,'into',num_splits-1,'segments of 10m + 1 remainder')
    
    start = 0
    audio_segments=[]
    for split in range(0,num_splits):
        end = delta_t*(split+1)
        audio_segments.append(data[start:end])
        start = end
        
    audio_segments.append(data[end:])
    
    #create list for segments
    clean_segments = []
    background_segments = []
    
    #initialize model
    print('initialising denoiser for ',filename)
    model = pretrained.dns64()
    
    # denoise each segment
    for ii,segment in enumerate(audio_segments):
        
        print('segment',ii,'/',num_splits)
        
        #create temp wav file
        temp_segment_file = mktemp('.wav')
        wavfile.write(temp_segment_file, raw_rate, segment)
        
        wav, rate = torchaudio.load(temp_segment_file)
        wav = convert_audio(wav, rate, model.sample_rate, model.chin)
        
        with torch.no_grad():
            denoised = model(wav[None])[0]
        clean_segments.append(denoised.data.numpy())
        background_segments.append(wav.data.numpy() - denoised.data.numpy())

    
    #stitch back together and save file in clean folder
    full_match_clean_audio = np.concatenate(clean_segments, axis=None)
    full_match_background_audio = np.concatenate(background_segments, axis=None)
    
    display(Audio(full_match_clean_audio,rate = model.sample_rate))
    display(Audio(full_match_background_audio,rate = model.sample_rate))
    
    wavfile.write(path_to_clean+filename, model.sample_rate, full_match_clean_audio)
    wavfile.write(path_to_background+filename, model.sample_rate, full_match_background_audio)
    
    
    
    

In [ ]:
# Last cell to run 

# from mutagen.mp3 import MP3
# from mutagen.wav import WAV

# audio = MP3("example.mp3")
# audio.export("example.wav", format=WAV)

def save_as_mp3_files(dir_path):
    # Get a list of all the files in the directory
    files = os.listdir(dir_path)

    # Iterate over the files
    for file in files:
        # Check if the file ends with '.mp3'
        if file.endswith('.mp3'):
            print(file)
            # Construct the full path to the file
            file_path = os.path.join(dir_path, file)

            # Use mutagen.File to open the file
            audio_file = mutagen.File(file_path)

            # Check if the file has a RIFF wrapper
            if audio_file.mime[0] == 'audio/wav':
                print('RIFF',audio_file,audio_file.mime)
                # If the file has a RIFF wrapper, remove it
                try:
                    audio_file = audio_file.mp3()
                except:
                    print('failed')
                    pass
                    

            # Use mutagen.File.save to save the file as an MP3
            audio_file.save(file_path[:-4]+'.wav')

            
            
            
import os
from pydub import AudioSegment

# Set the directory path to search

def save_as_mp3_files(directory):
    # Iterate through all files in the directory
    for filename in os.listdir(directory):
        # Check if the file ends with .mp3
        if filename.endswith('.mp3'):
            # Open the mp3 file
            mp3_file = AudioSegment.from_file(os.path.join(directory, filename))
            # Create the wav file name
            wav_filename = os.path.splitext(filename)[0] + ".wav"
            # Save the file as a wav file
            mp3_file.export(os.path.join(directory, wav_filename), format="wav")

            
save_as_mp3_files(path_to_mono) #1.58gb
save_as_mp3_files(path_to_clean) #1.14gb
save_as_mp3_files(path_to_background) #1.14gb

In [ ]:
# %%time
# model = pretrained.dns64()
# wav, sr = torchaudio.load('min10m.mp3')
# wav = convert_audio(wav, sr, model.sample_rate, model.chin)
# with torch.no_grad():
#     denoised = model(wav[None])[0]
# disp.display(disp.Audio(wav.data.numpy(), rate=model.sample_rate))
# disp.display(disp.Audio(denoised.data.numpy(), rate=model.sample_rate))

In [ ]:
# song = mp3_audio.set_channels(1)[0:60000]

# # Apply a low-pass filter to the audio
# filtered_song = song.low_pass_filter(5000)

# # Export the modified audio file
# filtered_song.export("filtered_song.mp3", format="mp3")

In [ ]:
# import noisereduce as nr

# # Load the audio file
# rate, data = wavfile.read(wname)
# noisy_t = (35*60+12)*1000+50000
# noisy_part = data[noisy_t:noisy_t+200000] 

# # Run the noise reduction algorithm
# noise_reduced_data = nr.reduce_noise(y=data, sr=rate,y_noise=noisy_part,n_fft=512)

# plt.specgram(noise_reduced_data, Fs=FS, NFFT=512, noverlap=0)  # plot
# plt.show()


In [ ]:
from pydub import AudioSegment

def cut_mp3(filepath, minutes, output_filepath):
  # read the MP3 file
  audio = AudioSegment.from_wav(filepath)

  # calculate the desired length in milliseconds
  desired_length_ms = minutes * 60 * 1000

  # get the first `desired_length_ms` milliseconds of the audio
  first_part = audio[:desired_length_ms]

  # save the first part to a new MP3 file
  first_part.export(output_filepath)

path = '../data/audio/clean/' + 'WOL_vs_FUL_2022-08-13.mp3'
length_num = 2
cut_mp3(path,length_num,'../data/audio/clean/'+str(length_num)+'min_test.wav')